<a href="https://colab.research.google.com/github/Pronow05/NL2SQL-LangChain/blob/main/llm_prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Postgresql DB Setup

In [1]:
!apt-get update

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,312 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,655 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,609 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.il

In [2]:
!apt-get install -y postgresql postgresql-contrib

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common
  postgresql-contrib ssl-cert sysstat
0 upgraded, 14 newly installed, 0 to remove and 21 not upgraded.
Need to get 18.4 MB of archives.
After this operation, 51.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubu

In [3]:
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [4]:
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD '006563';"

ALTER ROLE


In [5]:
!sudo -u postgres psql -c "CREATE DATABASE airbnb;"

CREATE DATABASE


In [6]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.7 MB/s eta 0:00:00


### Defining DB Schema

In [7]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="airbnb",
    user="postgres",
    password="006563",
    host="localhost"
)
cursor = conn.cursor()

# Check connection
cursor.execute("SELECT version();")
print(cursor.fetchone())

# Close cursor and connection
cursor.close()
conn.close()

('PostgreSQL 14.15 (Ubuntu 14.15-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [8]:
conn = psycopg2.connect(
    dbname="airbnb",
    user="postgres",
    password="006563",
    host="localhost"
)
cursor = conn.cursor()

# Create Listings Table
cursor.execute("""
CREATE TABLE listings (
    listing_id SERIAL PRIMARY KEY,
    name TEXT,
    host_id INTEGER,
    host_since DATE,
    host_location TEXT,
    host_response_time TEXT,
    host_response_rate TEXT,
    host_acceptance_rate TEXT,
    host_is_superhost BOOLEAN,
    host_total_listings_count INTEGER,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood TEXT,
    district TEXT,
    city TEXT,
    latitude DECIMAL(9,6),
    longitude DECIMAL(9,6),
    property_type TEXT,
    room_type TEXT,
    accommodates INTEGER,
    bedrooms INTEGER,
    amenities TEXT,
    price DECIMAL(10,2),
    minimum_nights INTEGER,
    maximum_nights INTEGER,
    review_scores_rating FLOAT,
    review_scores_accuracy FLOAT,
    review_scores_cleanliness FLOAT,
    review_scores_checkin FLOAT,
    review_scores_communication FLOAT,
    review_scores_location FLOAT,
    review_scores_value FLOAT,
    instant_bookable BOOLEAN
);

""")

# Create Reviews Table
cursor.execute("""
CREATE TABLE reviews (
    review_id SERIAL PRIMARY KEY,
    listing_id INTEGER REFERENCES listings(listing_id),
    date DATE,
    reviewer_id INTEGER
);
""")

conn.commit()
cursor.close()
conn.close()

### Data Import to Table from CSV

**Importing Data from Listings.csv to listings table using psql COPY command**

In [9]:
!sudo -u postgres psql -d airbnb

psql (14.15 (Ubuntu 14.15-0ubuntu0.22.04.1))
Type "help" for help.

airbnb=# COPY listings(     listing_id, name, host_id, host_since, host_location,     host_response_time, host_response_rate, host_acceptance_rate,     host_is_superhost, host_total_listings_count, host_has_profile_pic,     host_identity_verified, neighbourhood, district, city, latitude,     longitude, property_type, room_type, accommodates, bedrooms,     amenities, price, minimum_nights, maximum_nights, review_scores_rating,     review_scores_accuracy, review_scores_cleanliness, review_scores_checkin,     review_scores_communication, review_scores_location, review_scores_value,     instant_bookable ) FROM '/content/Listings.csv' WITH (FORMAT csv, HEADER true, ENCODING 'LATIN1');
COPY 279712
airbnb=# \q


**Cleaning Reviews.csv and then importing into reviews Table**

In [10]:
import pandas as pd

df_reviews = pd.read_csv("Reviews.csv", encoding="latin1")
df_reviews_clean = df_reviews.drop_duplicates(subset=['review_id'])
df_reviews_clean.to_csv("Reviews_clean.csv", index=False)

In [11]:
!sudo -u postgres psql -d airbnb

psql (14.15 (Ubuntu 14.15-0ubuntu0.22.04.1))
Type "help" for help.

airbnb=# COPY reviews(listing_id, review_id, date, reviewer_id) FROM '/content/Reviews_clean.csv' WITH (FORMAT csv, HEADER true, ENCODING 'LATIN1');
COPY 5372983
airbnb=# \q


**Verify Succesful Importing of Data**

In [12]:
conn = psycopg2.connect(
    dbname="airbnb",
    user="postgres",
    password="006563",
    host="localhost"
)
cursor = conn.cursor()

cursor.execute("SELECT * FROM listings LIMIT 5;")
print(cursor.fetchall())

cursor.execute("SELECT * FROM reviews LIMIT 5;")
print(cursor.fetchall())

cursor.close()
conn.close()

[(281420, 'Beautiful Flat in le Village Montmartre, Paris', 1466919, datetime.date(2011, 12, 3), 'Paris, Ile-de-France, France', None, None, None, False, 1, True, False, 'Buttes-Montmartre', None, 'Paris', Decimal('48.886680'), Decimal('2.333430'), 'Entire apartment', 'Entire place', 2, 1, '["Heating", "Kitchen", "Washer", "Wifi", "Long term stays allowed"]', Decimal('53.00'), 2, 1125, 100.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, False), (3705183, '39 mÃ\x82Â² Paris (Sacre CÃ\x85â\x80\x9cur)', 10328771, datetime.date(2013, 11, 29), 'Paris, Ile-de-France, France', None, None, None, False, 1, True, True, 'Buttes-Montmartre', None, 'Paris', Decimal('48.886170'), Decimal('2.345150'), 'Entire apartment', 'Entire place', 2, 1, '["Shampoo", "Heating", "Kitchen", "Essentials", "Washer", "Dryer", "Wifi", "Long term stays allowed"]', Decimal('120.00'), 2, 1125, 100.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, False), (4082273, 'Lovely apartment with Terrace, 60m2', 19252768, datetime.date(2014, 7, 31), 

# Setting Up Hugging Face for NL-SQL Conversion

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

import torch

hf_token = "hf_EUGYZywXLymYWRpLVKtbClxEhUmkUgUkI"
# Set your model ID (you can change this to your chosen model)
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create a Hugging Face text-to-text generation pipeline
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    max_length=256,
    token=hf_token
)

# Wrap the pipeline with LangChain’s HuggingFacePipeline LLM
llm = HuggingFacePipeline(pipeline=pipe)

**Using Groq API instead of Hugging Face due limited Computational Resourses**

In [13]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.6 MB/s eta 0:00:00


In [14]:
import os

os.environ["GROQ_API_KEY"] = "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"

In [16]:
from langchain_groq import ChatGroq

# Initialize the Groq LLM with the specified model
llm = ChatGroq(
    model_name="llama-3.2-3b-preview",
    temperature=0.3
)

#Leveraging LangChain for Prompting

In [17]:
from langchain.prompts import PromptTemplate

### Zero Shot Prompt Template

In [19]:
zero_shot_prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
    You are an expert SQL generator.

    Given the following database schema with comments:
    {schema}

    Generate a **syntactically correct** SQL query for the following question:
    **Question:** {question}

    **Rules:**
    - Ensure the query aligns with the schema.
    - Use appropriate SQL functions and joins as needed.
    - Return only the SQL query.

    SQL Query:
    """
)

### Few Shot Prompt Template

In [20]:
few_shot_prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
    You are an expert SQL generator.

    Given the following database schema with comments:
    {schema}

    Here are examples of correct SQL queries based on similar questions:

    **Example 1:**
    **Question:** "What is the total number of reviews for each listing?"
    **SQL:**
    ```
    SELECT listing_id, COUNT(review_id) AS total_reviews
    FROM reviews
    GROUP BY listing_id;
    ```

    **Example 2:**
    **Question:** "Find the average review score rating for each listing."
    **SQL:**
    ```
    SELECT listing_id, AVG(review_scores_rating) AS avg_rating
    FROM listings
    GROUP BY listing_id;
    ```

    **Example 3:**
    **Question:** "Get the number of listings hosted by each host."
    **SQL:**
    ```
    SELECT host_id, COUNT(listing_id) AS total_listings
    FROM listings
    GROUP BY host_id;
    ```

    **Example 4:**
    **Question:** "Retrieve the latest review date for each listing."
    **SQL:**
    ```
    SELECT listing_id, MAX(date) AS latest_review_date
    FROM reviews
    GROUP BY listing_id;
    ```

    Now, generate a SQL query for the following question:
    **Question:** {question}

    SQL Query:
    """
)

### Chain-Of-Thought Prompt Template

In [21]:
cot_prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
    You are an expert SQL generator. Before writing the SQL query, think step-by-step.

    Given the following database schema with comments:
    {schema}

    **Question:** {question}

    ### **Step 1: Understanding the Request**
    - Identify the key entities involved in the query.
    - Determine the relationships between tables.
    - Identify necessary SQL operations (JOIN, GROUP BY, COUNT, etc.).

    ### **Step 2: Breaking Down the Query Construction**
    - Identify the base table to query from.
    - Determine which columns should be selected.
    - Identify necessary filtering, aggregation, or sorting conditions.

    ### **Step 3: Constructing the SQL Query**
    - Use the above reasoning to generate the correct SQL statement.

    SQL Query:
    """
)

### Self-Consistency Prompt Template

In [22]:
self_consistency_prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
    You are an expert SQL generator.

    Given the following database schema with comments:
    {schema}

    Generate **three different candidate SQL queries** for the following question:
    **Question:** {question}

    **Candidate 1:**
    SQL Query:

    **Candidate 2:**
    SQL Query:

    **Candidate 3:**
    SQL Query:

    Now, analyze the three queries and select the best one based on correctness and efficiency.

    **Final Selected SQL Query:**
    """
)

In [18]:
from langchain.chains import LLMChain

In [23]:
# Function to generate SQL using a specified prompt template
def generate_sql(question, schema, prompt_template):
    chain = LLMChain(llm=llm, prompt=prompt_template)
    return chain.run({"schema": schema, "question": question})

In [24]:
schema_description = """
-- Database Schema for Airbnb Data

-- Table: listings
-- Description: Contains detailed information about each Airbnb listing.
-- Columns:
--   listing_id (BIGINT): Unique identifier for each listing.
--   name (TEXT): Title or name of the listing.
--   host_id (BIGINT): Unique identifier for the host of the listing.
--   host_since (DATE): Date when the host began hosting.
--   host_location (TEXT): Geographic location of the host.
--   host_response_time (TEXT): Typical response time by the host.
--   host_response_rate (TEXT): Host’s response rate (expressed as a percentage in text format).
--   host_acceptance_rate (TEXT): Host’s acceptance rate (expressed as a percentage in text format).
--   host_is_superhost (BOOLEAN): Indicates if the host is a superhost (true/false).
--   host_total_listings_count (BIGINT): Total number of listings managed by the host.
--   host_has_profile_pic (BOOLEAN): Indicates whether the host has a profile picture.
--   host_identity_verified (BOOLEAN): Indicates whether the host’s identity is verified.
--   neighbourhood (TEXT): Neighbourhood where the listing is located.
--   district (TEXT): District in which the listing is located.
--   city (TEXT): City where the listing is located.
--   latitude (DECIMAL): Latitude coordinate of the listing.
--   longitude (DECIMAL): Longitude coordinate of the listing.
--   property_type (TEXT): Type of property (e.g., apartment, house, etc.).
--   room_type (TEXT): Type of room offered (e.g., entire home, private room, shared room).
--   accommodates (INTEGER): Maximum number of guests the listing can accommodate.
--   bedrooms (INTEGER): Number of bedrooms in the listing.
--   amenities (TEXT): Comma-separated list of amenities provided.
--   price (DECIMAL): Price per night.
--   minimum_nights (INTEGER): Minimum number of nights required for booking.
--   maximum_nights (INTEGER): Maximum number of nights allowed for booking.
--   review_scores_rating (FLOAT): Overall rating score.
--   review_scores_accuracy (FLOAT): Accuracy rating score.
--   review_scores_cleanliness (FLOAT): Cleanliness rating score.
--   review_scores_checkin (FLOAT): Check-in rating score.
--   review_scores_communication (FLOAT): Communication rating score.
--   review_scores_location (FLOAT): Location rating score.
--   review_scores_value (FLOAT): Value for money rating score.
--   instant_bookable (BOOLEAN): Indicates whether the listing supports instant booking.

-- Table: reviews
-- Description: Contains reviews submitted for listings.
-- Columns:
--   review_id (SERIAL PRIMARY KEY): Unique identifier for each review.
--   listing_id (INTEGER): Foreign key linking to listings(listing_id), representing which listing was reviewed.
--   date (DATE): Date when the review was posted.
--   reviewer_id (INTEGER): Unique identifier for the reviewer.
"""

**Simple Query**

In [25]:
# Let the user input their natural language query
user_query = input("Enter your natural language SQL query: ")

# Generate SQL using different prompting techniques
print("🔹 Zero-Shot Output:")
print(generate_sql(user_query, schema_description, zero_shot_prompt))

print("\n🔹 Few-Shot Output:")
print(generate_sql(user_query, schema_description, few_shot_prompt))

print("\n🔹 Chain-of-Thought Output:")
print(generate_sql(user_query, schema_description, cot_prompt))

print("\n🔹 Self-Consistency Output:")
print(generate_sql(user_query, schema_description, self_consistency_prompt))

Enter your natural language SQL query: What is the average price per night for all listings in the database?
🔹 Zero-Shot Output:


<ipython-input-23-ef4ed8c6aa23>:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
<ipython-input-23-ef4ed8c6aa23>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain.run({"schema": schema, "question": question})


```sql
SELECT AVG(price) AS average_price_per_night
FROM listings;
```

This SQL query calculates the average price per night for all listings in the database by selecting the `price` column and applying the `AVG` function. The result is aliased as `average_price_per_night` for clarity.

🔹 Few-Shot Output:
    **SQL Query:**
    ```sql
SELECT AVG(price) AS avg_price_per_night
FROM listings;
```

    This SQL query calculates the average price per night for all listings in the database by using the `AVG()` function on the `price` column. The result is a single row with a single column, `avg_price_per_night`, containing the average price per night for all listings.

🔹 Chain-of-Thought Output:
### Step 1: Understanding the Request

- The key entity involved in the query is the "listings" table.
- The relationship between tables is that the "reviews" table has a foreign key referencing the "listings" table.
- The necessary SQL operations required are to select the average price per night f

**Little More complex Query**

In [26]:
# Let the user input their natural language query
user_query = input("Enter your natural language SQL query: ")

# Generate SQL using different prompting techniques
print("🔹 Zero-Shot Output:")
print(generate_sql(user_query, schema_description, zero_shot_prompt))

print("\n🔹 Few-Shot Output:")
print(generate_sql(user_query, schema_description, few_shot_prompt))

print("\n🔹 Chain-of-Thought Output:")
print(generate_sql(user_query, schema_description, cot_prompt))

print("\n🔹 Self-Consistency Output:")
print(generate_sql(user_query, schema_description, self_consistency_prompt))

Enter your natural language SQL query: For each property type, show the total number of listings and the average review score rating.
🔹 Zero-Shot Output:
```sql
SELECT 
    property_type,
    COUNT(*) AS total_listings,
    AVG(review_scores_rating) AS average_review_score
FROM 
    listings
GROUP BY 
    property_type
ORDER BY 
    property_type;
```

This SQL query generates the required output by grouping the listings by property type, counting the total number of listings for each type, and calculating the average review score rating for each type. The results are ordered by property type for easier reference.

🔹 Few-Shot Output:
  **SQL Query:**
  ```sql
SELECT 
  property_type, 
  COUNT(listing_id) AS total_listings, 
  AVG(review_scores_rating) AS avg_rating
FROM 
  listings
GROUP BY 
  property_type;
```

This SQL query will return the total number of listings for each property type and the average review score rating for each property type.

🔹 Chain-of-Thought Output:
### Step

# Evaluating the Generated SQL Queries


### Execute the Generated SQL Query

In [28]:
!pip install tabulate

In [30]:
import psycopg2
from tabulate import tabulate

def run_sql_query(query):
    try:
        # Connect to your PostgreSQL database
        conn = psycopg2.connect(
            dbname="airbnb",
            user="postgres",
            password="006563",  # Replace with your actual password
            host="localhost"
        )
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)
        results = cursor.fetchall()

        # Fetch column names
        colnames = [desc[0] for desc in cursor.description]

        # Print the results in a tabular format similar to SQL output
        print(tabulate(results, headers=colnames, tablefmt="psql"))

        cursor.close()
        conn.close()
        return results

    except Exception as e:
        print("Error executing query:", e)
        return None

# Example: Run a generated query
generated_query = "SELECT AVG(price) AS average_price FROM listings;"
results = run_sql_query(generated_query)


+-----------------+
|   average_price |
|-----------------|
|         608.793 |
+-----------------+


### Verify Correctness